In [1]:
import json
import subprocess

def get_video_duration(input_file):
    cmd = ['ffprobe', '-v', 'quiet', '-print_format', 'json', '-show_format', '-show_streams', input_file]
    result = subprocess.run(cmd, capture_output=True, text=True)
    data = json.loads(result.stdout)
    return float(data['format']['duration'])

In [2]:
import subprocess
import random
import cv2
import os
import math 
# Đường dẫn đến video
number_of_chunks = math.ceil(get_video_duration('sample_tv360.mp4')/get_video_duration('sample_tv360_highlight.mp4'))
for i in range(1, number_of_chunks+1):
    video_path = f'./data/sample_tv360/video_chunk/chunk_{i}.mp4'

    # Tính tổng thời gian của video
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration =  get_video_duration(video_path) 
    print("Đây là duration", duration)
    cap.release()

    # Chọn thời gian bắt đầu ngẫu nhiên (bảo đảm có đủ 25 giây)
    start_time = random.uniform(0, max(0, duration - math.ceil(duration/number_of_chunks)))
    start_time_formatted = f"{int(start_time // 3600):02}:{int((start_time % 3600) // 60):02}:{int(start_time % 60):02}"
    print(start_time_formatted)
    # Đường dẫn đến video đầu ra
    if not os.path.exists('./data/sample_tv360/gt'):
        os.makedirs('./data/sample_tv360/gt')
    output_path = f'./data/sample_tv360/gt/chunk_{i}_gt.mp4'

    # Lệnh ffmpeg để trích xuất video
    ffmpeg_cmd = [
        'ffmpeg',
        '-y',
        '-ss', start_time_formatted,  # Thời gian bắt đầu
        '-i', video_path,             # Video đầu vào
        '-to', f'{math.ceil(duration/number_of_chunks)}',                   # Thời lượng clip chính xác là 27 giây
        '-strict', 'experimental',    # Sử dụng codec âm thanh aac với tùy chọn strict
        '-shortest',                  # Đảm bảo rằng video sẽ kết thúc đúng khi đạt thời gian yêu cầu
        '-r', f'{fps}',
        output_path                   # Đường dẫn đầu ra
    ]

    # Thực thi lệnh ffmpeg
    subprocess.run(ffmpeg_cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print(f"Video được lưu mỗi {duration}s: {output_path}")


Đây là duration 721.052833
00:07:15
Video được lưu mỗi 721.052833s: ./data/sample_tv360/gt/chunk_1_gt.mp4
Đây là duration 721.043
00:05:15
Video được lưu mỗi 721.043s: ./data/sample_tv360/gt/chunk_2_gt.mp4
Đây là duration 721.058589
00:10:15
Video được lưu mỗi 721.058589s: ./data/sample_tv360/gt/chunk_3_gt.mp4
Đây là duration 721.0412
00:02:51
Video được lưu mỗi 721.0412s: ./data/sample_tv360/gt/chunk_4_gt.mp4
Đây là duration 721.033495
00:09:34
Video được lưu mỗi 721.033495s: ./data/sample_tv360/gt/chunk_5_gt.mp4
Đây là duration 721.04834
00:02:58
Video được lưu mỗi 721.04834s: ./data/sample_tv360/gt/chunk_6_gt.mp4
Đây là duration 721.030728
00:02:47
Video được lưu mỗi 721.030728s: ./data/sample_tv360/gt/chunk_7_gt.mp4
Đây là duration 721.04604
00:08:42
Video được lưu mỗi 721.04604s: ./data/sample_tv360/gt/chunk_8_gt.mp4
Đây là duration 721.037635
00:09:42
Video được lưu mỗi 721.037635s: ./data/sample_tv360/gt/chunk_9_gt.mp4


In [2]:
import os
import subprocess
from tqdm import tqdm

# Đường dẫn đến thư mục chứa các video cần nối
final_dir = './data/sample_tv360/gt'

# Tạo thư mục output nếu chưa tồn tại
final_output_dir = './data/sample_tv360/final'
if not os.path.exists(final_output_dir):
    os.makedirs(final_output_dir)

# Đường dẫn đến video sau khi nối
output_video = os.path.join(final_output_dir, 'complete_video_gt.mp4')
file_list_path = os.path.join(final_output_dir, 'merges_gt.txt')

# Tạo file danh sách các video để nối với tqdm
with open(file_list_path, 'w') as f:
    for video_file in sorted(os.listdir(final_dir), key=lambda x: int(x.split('_')[1])):
        file_path = os.path.join(final_dir, video_file)
        # Thêm đường dẫn chính xác vào file_list_path
        f.write(f"file '{os.path.abspath(file_path)}'\n")

# Kiểm tra nội dung file_list.txt
print("\nList of files to be concatenated:")
with open(file_list_path, 'r') as f:
    print(f.read())

# Lệnh ffmpeg để nối video, chuyển hướng stdout và stderr để ẩn thông báo từ ffmpeg
concat_cmd = [
    'ffmpeg',
    '-y',
    '-f', 'concat',
    '-safe', '0',
    '-i', file_list_path,
    '-c:v', 'libopenh264',  # Mã hóa lại video với libopenh264
    '-c:a', 'aac',          # Mã hóa lại audio với aac
    '-r', '30',
    '-strict', 'experimental',
    output_video
]

# Chạy lệnh nối video và tắt đầu ra của ffmpeg
print("Running ffmpeg to concatenate videos...")
subprocess.run(concat_cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Kiểm tra xem file output có được tạo thành công không
if os.path.exists(output_video):
    print(f"\nAll videos have been successfully concatenated into {output_video}")
else:
    print("\nError: The concatenation process failed. Please check the ffmpeg output for details.")



List of files to be concatenated:
file '/home/thiendc/projects/video_summarization/data/sample_tv360/gt/chunk_1_gt.mp4'
file '/home/thiendc/projects/video_summarization/data/sample_tv360/gt/chunk_2_gt.mp4'
file '/home/thiendc/projects/video_summarization/data/sample_tv360/gt/chunk_3_gt.mp4'
file '/home/thiendc/projects/video_summarization/data/sample_tv360/gt/chunk_4_gt.mp4'
file '/home/thiendc/projects/video_summarization/data/sample_tv360/gt/chunk_5_gt.mp4'
file '/home/thiendc/projects/video_summarization/data/sample_tv360/gt/chunk_6_gt.mp4'
file '/home/thiendc/projects/video_summarization/data/sample_tv360/gt/chunk_7_gt.mp4'
file '/home/thiendc/projects/video_summarization/data/sample_tv360/gt/chunk_8_gt.mp4'
file '/home/thiendc/projects/video_summarization/data/sample_tv360/gt/chunk_9_gt.mp4'
file '/home/thiendc/projects/video_summarization/data/sample_tv360/gt/chunk_10_gt.mp4'

Running ffmpeg to concatenate videos...

All videos have been successfully concatenated into ./data/sam

In [4]:
import subprocess

# Đường dẫn đến video
video1_path = "./data/sample_tv360/final/complete_video.mp4"
video2_path = "./data/sample_tv360/final/complete_video_gt.mp4"
output1_path = "./data/sample_tv360/final/complete_video_v2.mp4"
output2_path = "./data/sample_tv360/final/complete_video_gt_v2.mp4"

# Lấy thời lượng của hai video
def get_video_duration(video_path):
    result = subprocess.run(
        ["ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", video_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT
    )
    return float(result.stdout)

# Tính thời gian chung
duration1 = get_video_duration(video1_path)
duration2 = get_video_duration(video2_path)
common_duration = min(duration1, duration2)

# Cắt hai video sao cho có cùng độ dài
subprocess.run(["ffmpeg", "-y","-i", video1_path, "-t", str(common_duration), "-c", "copy", output1_path])
subprocess.run(["ffmpeg", "-y","-i", video2_path, "-t", str(common_duration), "-c", "copy", output2_path])


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 11.2.0 (Anaconda gcc)
  configuration: --prefix=/croot/ffmpeg_1722024521393/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_ --cc=/croot/ffmpeg_1722024521393/_build_env/bin/x86_64-conda-linux-gnu-cc --ar=/croot/ffmpeg_1722024521393/_build_env/bin/x86_64-conda-linux-gnu-ar --nm=/croot/ffmpeg_1722024521393/_build_env/bin/x86_64-conda-linux-gnu-nm --ranlib=/croot/ffmpeg_1722024521393/_build_env/bin/x86_64-conda-linux-gnu-ranlib --strip=/croot/ffmpeg_1722024521393/_build_env/bin/x86_64-conda-linux-gnu-strip --disable-doc --enable-swresample --enable-swscale --enable-openssl --enable-libxml2 --enable-libtheora --enable-demuxer=dash --enable-postproc --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfont

CompletedProcess(args=['ffmpeg', '-y', '-i', './data/sample_tv360/final/complete_video_gt.mp4', '-t', '270.066667', '-c', 'copy', './data/sample_tv360/final/complete_video_gt_v2.mp4'], returncode=0)

In [6]:
### Full version
import subprocess
import random
import cv2
import os
import math 
from tqdm import tqdm 
# Đường dẫn đến video
number_of_chunks = math.ceil(get_video_duration('sample_tv360.mp4')/get_video_duration('sample_tv360_highlight.mp4'))
chunk_duration = get_video_duration('sample_tv360_highlight.mp4')/number_of_chunks
video_path = "sample_tv360_highlight.mp4"
output_dir = "./data/sample_tv360/gt"
if os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)

for i in tqdm(range(number_of_chunks)):
    start_time = i * chunk_duration
    start_time_formatted = f"{int(start_time // 3600):02}:{int((start_time % 3600) // 60):02}:{int(start_time % 60):02}"
    
    output_path = os.path.join(output_dir, f"chunk_{i+1}_gt.mp4")

    ffmpeg_cmd = [
        'ffmpeg',
        '-y',
        '-ss', start_time_formatted,  # Thời gian bắt đầu
        '-i', video_path,             # Video đầu vào
        '-t', f'{chunk_duration}',    # Thời lượng mỗi đoạn
        '-c:v', 'copy',
        '-c:a', 'copy',
        output_path                   # Đường dẫn đầu ra
    ]

    subprocess.run(ffmpeg_cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print(f"Đã lưu video: {output_path}")


 22%|██▏       | 2/9 [00:00<00:01,  5.25it/s]

Đã lưu video: ./data/sample_tv360/gt/chunk_1_gt.mp4
Đã lưu video: ./data/sample_tv360/gt/chunk_2_gt.mp4


 44%|████▍     | 4/9 [00:01<00:01,  3.50it/s]

Đã lưu video: ./data/sample_tv360/gt/chunk_3_gt.mp4
Đã lưu video: ./data/sample_tv360/gt/chunk_4_gt.mp4


 56%|█████▌    | 5/9 [00:03<00:03,  1.15it/s]

Đã lưu video: ./data/sample_tv360/gt/chunk_5_gt.mp4


 78%|███████▊  | 7/9 [00:05<00:01,  1.08it/s]

Đã lưu video: ./data/sample_tv360/gt/chunk_6_gt.mp4
Đã lưu video: ./data/sample_tv360/gt/chunk_7_gt.mp4


100%|██████████| 9/9 [00:05<00:00,  1.60it/s]

Đã lưu video: ./data/sample_tv360/gt/chunk_8_gt.mp4
Đã lưu video: ./data/sample_tv360/gt/chunk_9_gt.mp4
